In [17]:
pip install langchain pypdf openai chromadb tiktoken docx2txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=06804cbeaa76fa12a6370c5438ef741103908b548cb96f1c0d5719f53b6792e9
  Stored in directory: /Users/ali/Library/Caches/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import PyPDFLoader

In [7]:
pdf_loader = PyPDFLoader("./Downloads/BN-Article.pdf")

In [31]:
documents = pdf_loader.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(documents)

In [16]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import os


In [33]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY))


In [34]:
question = "Who wrote this article?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [36]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0, openai_api_key = OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': 'Who wrote this article?',
 'result': 'The author of this article is B. Honari.'}